# 4.1. Manually Consuming an Iterator

In [9]:
# To manually consume an iterable, use the next() function and write your code to catch the StopIteration exception
with open('test.txt') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration: # signal the end of iteration 
        pass

with open('test.txt') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')

items = [1, 2, 3]
# Get the iterator 
it = iter(items) # Invokes  items.__iter__() 
# Run the iterator 
next(it)
next(it)
next(it)


A
B
C
D
E
F
G
A
B
C
D
E
F
G


3

# 4.2. Delegating Iteration

In [16]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = [] 
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self): # __iter__ method simply forwards the iteration request to the internally held_children attribute.
        return iter(self._children)

# Example
if __name__ == '__main__':
    root = Node(0)
    childA = Node(1)
    childB = Node(2)
    root.add_child(childA)
    root.add_child(childB)
    
    for ch in root:
        print(ch)
    # Outputs Node(1), Node(2)
    

Node(1)
Node(2)


# 4.3. Creating New Iteration Patterns with Generators

In [24]:
# Create a new Kind of iteration pattern
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x 
        x += increment
        
for n in frange(0, 4, 0.5):
    print(n)

list(frange(0, 1, 0.125))

# a generator only runs in response to iteration
def countdown(n):
    print('Staring to count from', n)
    while n > 0:
        yield n 
        n -= 1
    print('Done!')

# Create the generator, notice no output appears
c = countdown(3)

# Run to first yield and emit a value 
next(c)
# Run to the next yield
next(c)
# Run to next yield (iteration stops)
next(c)
next(c)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
Staring to count from 3
Done!


StopIteration: 

# 4.4. Implementing the Iterator Protocol 

In [27]:
class Node:
    def __init__(self, value):
        self._value = value 
        self._children = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self): # Python's iterator protocol requires __iter__() to return a special iterator object
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

# Example 
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)
        

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


# 4.5. Iterating in Reverse

In [29]:
# iterate in reverse over a sequence 
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

# print a file backwards
f = open('test.txt')
# Be aware that turning an iterable into a list as shown could consyme a lot of memory 
for line in reversed(list(f)):
    print(line, end='')

class Countdown:
    def __init__(self, start):
        self.start = start
    
    # Forward iterator 
    def __iter__(self):
        n = self.start 
        while n > 0:
            yield n 
            n -= 1
    
    # Reverse iterator 
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n 
            n += 1 



4
3
2
1
G
F
E
D
C
B
A


# 4.6.Defining Generator Functions with Extra Sate

In [34]:
from collections import deque 

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
    
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line 
    
    def clear(self):
        self.history.clear()

with open('test.txt') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')


# 4.7. Taking a Slice of an Iterator

In [37]:
# itertools.islice() function is perfectly suited for taking slices of iterators and generators 
def count(n):
    while True:
        yield n 
        n += 1 

c = count(0)

# Now using islice() 
import itertools
for x in itertools.islice(c, 10, 20):
    print(x)

# no information is known about their length 

10
11
12
13
14
15
16
17
18
19


# 4.8. Skipping the First Part of an Iterable

In [46]:
with open('test.txt') as f:
    for line in f:
        print(line, end='')

from itertools import dropwhile
with open('test.txt') as f:
    for line in dropwhile(lambda line: line.startswith('#'), f):
        print(line, end='')

from itertools import islice 
items = ['a', 'b', 'c', 1, 4, 10, 15]
for x in islice(items, 3, None):
    print(x)

#A
#B
#C
D
E
F
G
D
E
F
G
1
4
10
15


# 4.9. Iterating Over All Possible Combinations or Permutations

In [57]:
items = ['a', 'b', 'c']
from itertools import permutations # takes a collection of items and produces a sequence of tuples that rearranges all of the items into all possible permutations
for p in permutations(items):
    print(p)

for p in permutations(items, 2): # permutations of a smaller length
    print(p)

# itertools.combinations() to produce a sequence of combinations of items taken from the input
from itertools import combinations
for c in combinations(items, 3):
    print(c)

for c in combinations(items, 2):
    print(c)

for c in combinations(items, 1):
    print(c)

from itertools import combinations_with_replacement 
for c in combinations_with_replacement(items, 3):  # The itertools.combinations_with_replacement() function relaxes this, and allows the same item to be chosen more than once
    print(c)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')
('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')
('a', 'b', 'c')
('a', 'b')
('a', 'c')
('b', 'c')
('a',)
('b',)
('c',)
('a', 'a', 'a')
('a', 'a', 'b')
('a', 'a', 'c')
('a', 'b', 'b')
('a', 'b', 'c')
('a', 'c', 'c')
('b', 'b', 'b')
('b', 'b', 'c')
('b', 'c', 'c')
('c', 'c', 'c')


# 4.10. Iterating Over the Index-Value Pairs of a Sequence

In [61]:
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx, val)

# printing output with canonical line numbers
for idx, val in enumerate(my_list, 1):
    print(idx, val)

def parse_data(filename):
    with open(filename, 'rt') as f:
        for lineno, line in enumerate(f,1):
            fields = line.split()
            try:
                count = int(fields[1])
            except ValueError as e:
                print('Line {}: Parse error: {}'.format(lineno, e))

0 a
1 b
2 c
1 a
2 b
3 c


# 4.11. iterating Over Multiple Sequences Simultaneously

In [67]:
xpts = [1, 5, 4, 2, 10, 7]
ypts = [101, 78, 37, 15, 62, 99]
# iterate over more than one sequence simultaneously 
for x, y in zip(xpts, ypts): # zip() is commonly used whenever you need to pair data together
    print(x,y)

a = [1, 2, 3]
b = ['w', 'x', 'y', 'z']
for i in zip(a,b): # the length of the iteration is the same as the length of the shortest input
    print(i)

from itertools import zip_longest
for i in zip_longest(a,b): # 
    print(i)

for i in zip_longest(a,b, fillvalue=0):
    print(i)

1 101
5 78
4 37
2 15
10 62
7 99
(1, 'w')
(2, 'x')
(3, 'y')
(1, 'w')
(2, 'x')
(3, 'y')
(None, 'z')
(1, 'w')
(2, 'x')
(3, 'y')
(0, 'z')


# 4.12. Iterating on Items in Separate Containers

In [70]:
# itertools.chain() accepts one or more iterables as arguments. 
from itertools import chain
a = [1, 2, 3, 4]
b = ['x', 'y', 'z']
for x in chain(a, b):
    print(x)

1
2
3
4
x
y
z


# 4.13. Creating Data Processing Pipelines

In [ ]:
import os 
import fnmatch 
import gzip
import bz2
import re 

def gen_find(filepat, top):
    '''
    Find all filenames in a directory tree that match a shell wildcard pattern 
    '''
    for path, dirlist, filelist in os.walk(top):
        for name in fnmatch.filter(filelist, filepat):
            yield os.path.join(path, name)
